In [1]:
import os
import pandas as pd
import numpy as np
import PIL
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import load_model,save_model
from keras.callbacks import ModelCheckpoint
from matplotlib.patches import Rectangle
import os
from scipy.misc import imsave

import keras

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam,Adagrad
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from math import sqrt
from keras.callbacks import History 
from keras.optimizers import Adam, SGD
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,Input
from keras.layers import BatchNormalization
from keras.models import Model
from keras.activations import relu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from math import sqrt
from keras.callbacks import History 
import gc

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
base_dir = '/home/roshansanthosh/Documents/YOLO/YOLO/MultiClass'

In [3]:
from keras.preprocessing import image

def read_img(filepath, size):
    img = image.load_img((filepath), target_size=size)
    img = image.img_to_array(img,data_format='channels_last')
    return img

os.chdir(base_dir + '/25x25/')

X = []

for i in range(8000):
    img = read_img(str(i)+'.png',(25,25))
    X.append(img)
    
    


truth = pd.read_csv('25x25_truth.csv')
y = np.array(truth)
y = np.array(y[:,1:])


X= np.array(X)
y = np.array(y)


train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size=25


trainX = X[:7000,:,:,:]
trainy = y[:7000,:]

testX = X[7000:,:,:,:]
testy = y[7000:,:]

train_generator = train_datagen.flow(x=trainX,y=trainy,batch_size=batch_size)
test_generator = test_datagen.flow(x=testX,y=testy,batch_size=batch_size)


def getIoU(model,testData,testTruth,numAnchors = 2,rescale=True):
    
    anchorLength = int(testTruth[0].shape[0]/numAnchors)
 
    
    sample = testData
    if rescale==True:
        sample = testData/255.
    preds = model.predict(sample)
    
    IoUSum = 0.0
    for i in range(len(testTruth)):
        iouTotal = 0.0
        for j in range(0,len(testTruth[i]),anchorLength):
            truth = testTruth[i][j:(j+4)]
            pred  =  preds[i][j:(j+4)]
            iouTotal += bb_intersection_over_union(truth,pred)
        
        IoUSum += iouTotal/numAnchors
    
    meanIoU = IoUSum/len(testTruth)
    
    return(meanIoU)
   

In [4]:
model = load_model(base_dir + '/Models/model_3_51_0.0078.hdf5')

In [5]:
model1 = load_model(base_dir + '/Models/model_3_36_0.0078.hdf5')

In [6]:
from IPython.display import display

In [6]:
def testSample(obs,modelList,numAnchors = 2):
    
    if obs>=len(testX):
        print("Out of bounds")
        
    sample = testX[obs]
    
    sample = sample.reshape((1,25,25,3))
    sample = sample/255.
    truth = [testy[obs]]
    
    IoU1 = getIoU(modelList[0][0], sample, truth,rescale=False)
    IoU2 = getIoU(modelList[1][0], sample, truth,rescale=False)
    
    pred1 = modelList[0][0].predict(sample)
    pred2 = modelList[1][0].predict(sample)
    
    
    softmax1 = getLabelProb(pred1,truth)
    softmax2 = getLabelProb(pred2,truth)
    
    factor=0.2
    matplotlib.rcParams['figure.figsize'] = [40*factor,75*factor]
    
    
    fig,ax = plt.subplots(1)
    
    ax.imshow(sample[0,:,:,:])
    
    rectList1 = addRect(anchorLength,boxLimits,pred1,modelList[0][1])
    rectList2 = addRect(anchorLength,boxLimits,pred2,modelList[1][1])
    
    for rec in rectList1:
        ax.add_patch(rec)
    for rec in rectList2:
        ax.add_patch(rec)
        

    
    print("Truth : {}".format(truth[0]))
    print("Pred1 : {}".format(pred1))
    print("Pred2 : {}".format(pred2))
    
    print("IoU1 : {} IoU2 : {}".format(IoU1,IoU2))
    print("Softmax Label 1 : {}".format(softmax1))
    print("Softmax Label 2 : {}".format(softmax2))
    
    display(truth[0])
    display(pred1)
    display(pred2)
    
    
    plt.show()
    

In [ ]:
testSample(91,[[model,'red'],[model1,'blue']])